In [ ]:
# ============================================================================
# HANDS-ON EXERCISE: OVERFITTING vs UNDERFITTING
# Датасет: Titanic (Kaggle)
# ============================================================================

# ============================================================================
# КРОК 1: ІМПОРТ БІБЛІОТЕК
# ============================================================================
print("📚 Завантажуємо необхідні бібліотеки...")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Налаштування для гарних графіків
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Бібліотеки завантажено успішно!\n")

In [ ]:
# ============================================================================
# КРОК 2: ЗАВАНТАЖЕННЯ ТА ПІДГОТОВКА ДАТАСЕТУ TITANIC
# https://www.kaggle.com/datasets/dbdmobile/tita111
# ============================================================================
print("🚢 Завантажуємо датасет Titanic...")

# Завантажуємо датасет з Kaggle (можна також використати seaborn)
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

print(f"✅ Датасет завантажено! Кількість записів: {len(df)}")
print(f"Перші 5 рядків датасету:\n")
print(df.head())
print("\n" + "="*80 + "\n")

In [ ]:
# ============================================================================
# КРОК 3: ПІДГОТОВКА ДАНИХ
# ============================================================================
print("🔧 Підготовка даних...\n")

# Вибираємо важливі колонки
df_clean = df[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']].copy()

# Заповнюємо пропущені значення віку медіаною - ВИПРАВЛЕНО
df_clean = df_clean.fillna({'Age': df_clean['Age'].median()})

# Перетворюємо стать на числа (Male=1, Female=0)
le = LabelEncoder()
df_clean['Sex'] = le.fit_transform(df_clean['Sex'])

# Видаляємо рядки з пропущеними значеннями
df_clean = df_clean.dropna()

print(f"✅ Дані підготовлено! Залишилось {len(df_clean)} записів")
print(f"Ознаки для навчання: Pclass, Sex, Age, SibSp, Parch, Fare")
print(f"Цільова змінна: Survived (0 = загинув, 1 = вижив)\n")
print("="*80 + "\n")

In [ ]:
# ============================================================================
# ЧАСТИНА 1: OVERFITTING (ПЕРЕНАВЧАННЯ)
# ============================================================================
print("🔴 ЧАСТИНА 1: ДЕМОНСТРАЦІЯ OVERFITTING")
print("="*80)
print("Що ми робимо:")
print("- Берємо ДУЖЕ МАЛО даних (тільки 50 прикладів)")
print("- Навчаємо СКЛАДНУ модель (глибоке дерево рішень, max_depth=15)")
print("- Модель 'запам'ятає' всі приклади, але не зможе узагальнити\n")

# Беремо тільки 50 перших записів для overfitting
df_small = df_clean.head(50).copy()

# Розділяємо на ознаки (X) та цільову змінну (y)
X_small = df_small.drop('Survived', axis=1)
y_small = df_small['Survived']

# Розділяємо на train та test (60% train, 40% test)
X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(
    X_small, y_small, test_size=0.4, random_state=42
)

print(f"📊 Розмір тренувального набору: {len(X_train_small)} записів")
print(f"📊 Розмір тестового набору: {len(X_test_small)} записів\n")

# Навчаємо ДУЖЕ СКЛАДНУ модель (глибоке дерево)
model_overfit = DecisionTreeClassifier(max_depth=15, min_samples_split=2, random_state=42)
model_overfit.fit(X_train_small, y_train_small)

# Передбачаємо результати
y_train_pred_overfit = model_overfit.predict(X_train_small)
y_test_pred_overfit = model_overfit.predict(X_test_small)

# Обчислюємо точність
train_accuracy_overfit = accuracy_score(y_train_small, y_train_pred_overfit)
test_accuracy_overfit = accuracy_score(y_test_small, y_test_pred_overfit)

print("📈 РЕЗУЛЬТАТИ OVERFITTING:")
print(f"   Точність на тренувальних даних: {train_accuracy_overfit*100:.1f}%")
print(f"   Точність на тестових даних: {test_accuracy_overfit*100:.1f}%")
print(f"   ⚠️ Різниця: {(train_accuracy_overfit - test_accuracy_overfit)*100:.1f}%")
print("\n💡 ПОЯСНЕННЯ:")
print("   Модель ПЕРЕНАВЧИЛАСЬ! Вона 'запам'ятала' тренувальні дані,")
print("   але погано працює на нових даних (тестових).")
print("   Це класичний приклад OVERFITTING.\n")
print("="*80 + "\n")

In [ ]:
# ============================================================================
# ЧАСТИНА 2: UNDERFITTING (НЕДОНАВЧАННЯ)
# ============================================================================
print("🔵 ЧАСТИНА 2: ДЕМОНСТРАЦІЯ UNDERFITTING")
print("="*80)
print("Що ми робимо:")
print("- Беремо всі дані")
print("- Використовуємо ЛИШЕ ОДНУ ПОГАНУ ОЗНАКУ (PassengerId)")
print("- Модель не може навчитись, бо ознака не пов'язана з виживанням\n")

# Підготовка даних з поганою ознакою
df_bad = df[['Survived', 'PassengerId']].copy()
df_bad = df_bad.dropna()

X_bad = df_bad[['PassengerId']]
y_bad = df_bad['Survived']

# Розділяємо на train та test
X_train_bad, X_test_bad, y_train_bad, y_test_bad = train_test_split(
    X_bad, y_bad, test_size=0.3, random_state=42
)

print(f"📊 Розмір тренувального набору: {len(X_train_bad)} записів")
print(f"📊 Розмір тестового набору: {len(X_test_bad)} записів")
print(f"📊 Використовуємо ознаку: PassengerId (не корелює з виживанням!)\n")

# Навчаємо модель на поганій ознаці
model_underfit = DecisionTreeClassifier(max_depth=3, random_state=42)
model_underfit.fit(X_train_bad, y_train_bad)

# Передбачаємо результати
y_train_pred_underfit = model_underfit.predict(X_train_bad)
y_test_pred_underfit = model_underfit.predict(X_test_bad)

# Обчислюємо точність
train_accuracy_underfit = accuracy_score(y_train_bad, y_train_pred_underfit)
test_accuracy_underfit = accuracy_score(y_test_bad, y_test_pred_underfit)

print("📈 РЕЗУЛЬТАТИ UNDERFITTING:")
print(f"   Точність на тренувальних даних: {train_accuracy_underfit*100:.1f}%")
print(f"   Точність на тестових даних: {test_accuracy_underfit*100:.1f}%")
print(f"   Різниця: {abs(train_accuracy_underfit - test_accuracy_underfit)*100:.1f}%")
print("\n💡 ПОЯСНЕННЯ:")
print("   Модель НЕДОНАВЧИЛАСЬ! PassengerId не має зв'язку з виживанням,")
print("   тому модель погано працює і на тренувальних, і на тестових даних.")
print("   Це класичний приклад UNDERFITTING через погані ознаки.\n")
print("="*80 + "\n")

In [ ]:
# ============================================================================
# ЧАСТИНА 3: GOOD FIT (ОПТИМАЛЬНА МОДЕЛЬ)
# ============================================================================
print("🟢 ЧАСТИНА 3: ДЕМОНСТРАЦІЯ GOOD FIT (ОПТИМАЛЬНА МОДЕЛЬ)")
print("="*80)
print("Що ми робимо:")
print("- Беремо всі дані")
print("- Використовуємо ХОРОШІ ОЗНАКИ (Pclass, Sex, Age, Fare)")
print("- Навчаємо модель ОПТИМАЛЬНОЇ СКЛАДНОСТІ (max_depth=5)")
print("- Модель добре узагальнює та працює на нових даних\n")

# Використовуємо весь датасет з хорошими ознаками
X_full = df_clean.drop('Survived', axis=1)
y_full = df_clean['Survived']

# Розділяємо на train та test
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
    X_full, y_full, test_size=0.3, random_state=42
)

print(f"📊 Розмір тренувального набору: {len(X_train_full)} записів")
print(f"📊 Розмір тестового набору: {len(X_test_full)} записів\n")

# Навчаємо модель оптимальної складності
model_goodfit = DecisionTreeClassifier(max_depth=5, min_samples_split=20, random_state=42)
model_goodfit.fit(X_train_full, y_train_full)

# Передбачаємо результати
y_train_pred_goodfit = model_goodfit.predict(X_train_full)
y_test_pred_goodfit = model_goodfit.predict(X_test_full)

# Обчислюємо точність
train_accuracy_goodfit = accuracy_score(y_train_full, y_train_pred_goodfit)
test_accuracy_goodfit = accuracy_score(y_test_full, y_test_pred_goodfit)

print("📈 РЕЗУЛЬТАТИ GOOD FIT:")
print(f"   Точність на тренувальних даних: {train_accuracy_goodfit*100:.1f}%")
print(f"   Точність на тестових даних: {test_accuracy_goodfit*100:.1f}%")
print(f"   Різниця: {abs(train_accuracy_goodfit - test_accuracy_goodfit)*100:.1f}%")
print("\n💡 ПОЯСНЕННЯ:")
print("   Модель ОПТИМАЛЬНА! Вона добре працює на обох наборах даних.")
print("   Різниця між train та test мінімальна.")
print("   Це приклад GOOD FIT - ідеального балансу.\n")
print("="*80 + "\n")

In [ ]:
# ============================================================================
# ВІЗУАЛІЗАЦІЯ: ПОРІВНЯННЯ ВСІХ ТРЬОХ МОДЕЛЕЙ
# ============================================================================
print("📊 СТВОРЮЄМО ВІЗУАЛІЗАЦІЮ ДЛЯ ПОРІВНЯННЯ...\n")

# Створюємо фігуру з трьома графіками
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Порівняння: Overfitting vs Underfitting vs Good Fit',
             fontsize=16, fontweight='bold', y=1.02)

# Дані для графіків
models_data = [
    {
        'title': 'OVERFITTING\n(Перенавчання)',
        'train': train_accuracy_overfit * 100,
        'test': test_accuracy_overfit * 100,
        'color': '#e74c3c',
        'params': 'Мало даних (50)\nГлибоке дерево (depth=15)'
    },
    {
        'title': 'UNDERFITTING\n(Недонавчання)',
        'train': train_accuracy_underfit * 100,
        'test': test_accuracy_underfit * 100,
        'color': '#3498db',
        'params': 'Багато даних\nПогана ознака (PassengerId)'
    },
    {
        'title': 'GOOD FIT\n(Оптимально)',
        'train': train_accuracy_goodfit * 100,
        'test': test_accuracy_goodfit * 100,
        'color': '#2ecc71',
        'params': 'Багато даних\nХороші ознаки + depth=5'
    }
]

# Малюємо кожен графік
for i, (ax, data) in enumerate(zip(axes, models_data)):
    categories = ['Train', 'Test']
    values = [data['train'], data['test']]

    # Малюємо стовпчики окремо з різною прозорістю
    bar1 = ax.bar([0], [values[0]], color=data['color'], alpha=0.9, width=0.6, label='Train')
    bar2 = ax.bar([1], [values[1]], color=data['color'], alpha=0.5, width=0.6, label='Test')

    # Додаємо значення на стовпчики
    ax.text(0, values[0] + 1, f'{values[0]:.1f}%',
            ha='center', va='bottom', fontsize=12, fontweight='bold')
    ax.text(1, values[1] + 1, f'{values[1]:.1f}%',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

    ax.set_title(data['title'], fontsize=13, fontweight='bold', pad=10)
    ax.set_ylabel('Точність (%)', fontsize=11)
    ax.set_ylim(0, 105)
    ax.set_xticks([0, 1])
    ax.set_xticklabels(categories)
    ax.grid(axis='y', alpha=0.3, linestyle='--')

    # Додаємо параметри моделі
    ax.text(0.5, -0.25, data['params'],
            transform=ax.transAxes, ha='center', fontsize=9,
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

plt.tight_layout()
plt.savefig('overfitting_underfitting_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Графік створено та збережено як 'overfitting_underfitting_comparison.png'\n")
print("="*80 + "\n")

In [ ]:
# ============================================================================
# ПІДСУМКОВА ТАБЛИЦЯ РЕЗУЛЬТАТІВ
# ============================================================================
print("📋 ПІДСУМКОВА ТАБЛИЦЯ РЕЗУЛЬТАТІВ:")
print("="*80 + "\n")

# Створюємо красиву таблицю з кольоровим кодуванням
results_data = [
    ['OVERFITTING', train_accuracy_overfit, test_accuracy_overfit,
     abs(train_accuracy_overfit - test_accuracy_overfit)],
    ['UNDERFITTING', train_accuracy_underfit, test_accuracy_underfit,
     abs(train_accuracy_underfit - test_accuracy_underfit)],
    ['GOOD FIT', train_accuracy_goodfit, test_accuracy_goodfit,
     abs(train_accuracy_goodfit - test_accuracy_goodfit)]
]

# Виводимо таблицю
print(f"{'Тип моделі':<20} {'Train Accuracy':<18} {'Test Accuracy':<18} {'Різниця':<12} {'Оцінка'}")
print("-" * 95)

for i, (model_type, train_acc, test_acc, diff) in enumerate(results_data):
    # Визначаємо оцінку
    if i == 0:  # Overfitting
        grade = "ПОГАНО (перенавчання)"
        symbol = "[X]"
    elif i == 1:  # Underfitting
        grade = "ПОГАНО (недонавчання)"
        symbol = "[X]"
    else:  # Good fit
        grade = "ВІДМІННО!"
        symbol = "[✓]"

    print(f"{model_type:<20} {train_acc*100:>6.1f}%{'':<10} {test_acc*100:>6.1f}%{'':<10} "
          f"{diff*100:>5.1f}%{'':<6} {symbol} {grade}")

print("\n" + "="*80)
print("\n💡 ЯК ЧИТАТИ ТАБЛИЦЮ:")
print("   • Train Accuracy - точність на тренувальних даних")
print("   • Test Accuracy - точність на нових (тестових) даних")
print("   • Різниця - різниця між train та test (чим менше, тим краще)")
print("\n🎯 ІДЕАЛЬНА МОДЕЛЬ: висока test accuracy + мала різниця між train та test\n")
print("="*80 + "\n")

In [ ]:
# ============================================================================
# ДОМАШНЄ ЗАВДАННЯ
# ============================================================================
print("📝 ДОМАШНЄ ЗАВДАННЯ:")
print("="*80)
print("""
ЗАВДАННЯ: Знайдіть оптимальне значення max_depth для Decision Tree (максимальна глибина дерева рішень)

ЩО ЦЕ ОЗНАЧАЄ:

Дерево рішень працює як серія питань: "Чи стать = жінка?", "Чи вік > 30?", "Чи клас = 1?"
Глибина - це кількість питань підряд, які модель може задати
max_depth=3 означає максимум 3 питання підряд
max_depth=15 означає максимум 15 питань підряд

Візуально:
max_depth=1 (дуже просто):
└─ Чи стать = жінка?
    ├─ Так → Вижив
    └─ Ні → Загинув
max_depth=3 (оптимально):
└─ Чи стать = жінка?
    ├─ Так → Чи клас = 3?
    │   ├─ Так → Загинув
    │   └─ Ні → Вижив
    └─ Ні → Чи вік < 10?
        ├─ Так → Вижив
        └─ Ні → Загинув
max_depth=15 (занадто складно):
└─ Чи стать = жінка?
    ├─ Так → Чи клас = 3?
    │   ├─ Так → Чи вік > 25?
    │   │   ├─ Так → Чи SibSp = 1?
    │   │   │   ├─ Так → ... (ще 11 рівнів!)


ЩО РОБИТИ:
1. Запустіть цю комірку
2. Подивіться на результати та графік
3. Змініть значення max_depth_to_test (наприклад, на 3, 7, 10, 15)
4. Запустіть ще раз і порівняйте результати
5. Знайдіть таке значення max_depth, де:
   - Test Accuracy найвища
   - Різниця між Train та Test мінімальна

ДАЙТЕ ВІДПОВІДІ НА ПИТАННЯ:
- Яке значення max_depth найкраще?
- Що відбувається коли max_depth занадто великий (наприклад, 15)?
- Що відбувається коли max_depth занадто малий (наприклад, 2)?
""")
print("="*80 + "\n")

# ============================================================================
# ЕКСПЕРИМЕНТ: Тестування різних значень max_depth
# ============================================================================

print("🔧 Підготовка даних для експерименту...\n")

# Використовуємо ВЕСЬ очищений датасет для стабільних результатів
X_hw = df_clean.drop('Survived', axis=1)
y_hw = df_clean['Survived']

# Розділяємо на train (70%) та test (30%)
X_train_hw, X_test_hw, y_train_hw, y_test_hw = train_test_split(
    X_hw, y_hw, test_size=0.3, random_state=42
)

print(f"📊 Використовуємо: {len(X_train_hw)} записів для навчання, {len(X_test_hw)} для тесту")
print(f"   (Використовуємо весь датасет для стабільних результатів)\n")

# ========================================
# ЗМІНІТЬ ЦЕ ЗНАЧЕННЯ ДЛЯ ЕКСПЕРИМЕНТУ:
max_depth_to_test = 1  # ← Спробуйте: 1, 2, 3, 5, 7, 10, 15, 20
# ========================================

print(f"🔬 Тестуємо модель з max_depth = {max_depth_to_test}\n")

# Навчаємо модель
model_hw = DecisionTreeClassifier(max_depth=max_depth_to_test, min_samples_split=10, random_state=42)
model_hw.fit(X_train_hw, y_train_hw)

# Обчислюємо точність
train_acc_hw = accuracy_score(y_train_hw, model_hw.predict(X_train_hw))
test_acc_hw = accuracy_score(y_test_hw, model_hw.predict(X_test_hw))
diff_hw = abs(train_acc_hw - test_acc_hw)

# Виводимо результати
print("="*60)
print(f"РЕЗУЛЬТАТИ для max_depth = {max_depth_to_test}:")
print("="*60)
print(f"Train Accuracy:  {train_acc_hw*100:.1f}%")
print(f"Test Accuracy:   {test_acc_hw*100:.1f}%")
print(f"Різниця:         {diff_hw*100:.1f}%")
print("="*60)

# Висновок
print("\n💡 ВИСНОВОК:")
if max_depth_to_test is not None and max_depth_to_test <= 2:
    print("❌ UNDERFITTING! Модель занадто проста")
    print(f"   Train accuracy = {train_acc_hw*100:.1f}%, Test accuracy = {test_acc_hw*100:.1f}%")
    print("   → Модель не може вловити складні закономірності")
    print("   → Обидва показники невисокі")
    print("   → Спробуйте БІЛЬШИЙ max_depth")
elif max_depth_to_test is not None and max_depth_to_test >= 12:
    print("❌ OVERFITTING! Модель занадто складна")
    print(f"   Train = {train_acc_hw*100:.1f}%, Test = {test_acc_hw*100:.1f}%")
    print(f"   Різниця = {diff_hw*100:.1f}% - це занадто багато!")
    print("   → Модель 'запам'ятала' тренувальні дані")
    print("   → Спробуйте МЕНШИЙ max_depth")
elif max_depth_to_test is not None and 4 <= max_depth_to_test <= 8:
    print("✅ GOOD FIT! Модель добре збалансована")
    print(f"   Train accuracy = {train_acc_hw*100:.1f}%, Test accuracy = {test_acc_hw*100:.1f}%")
    print(f"   Різниця = {diff_hw*100:.1f}% - це прийнятно!")
    print("   → Це оптимальний баланс між складністю і узагальненням!")
    print("   → Модель добре працює на нових даних")
elif max_depth_to_test == 3:
    if train_acc_hw < 0.78:
        print("⚠️ Ще трохи UNDERFITTING")
    else:
        print("✅ Непогано! Майже оптимально")
elif max_depth_to_test is not None and 9 <= max_depth_to_test <= 11:
    if diff_hw > 0.08:
        print("⚠️ Починається OVERFITTING")
        print(f"   Різниця {diff_hw*100:.1f}% вже помітна")
    else:
        print("✅ Ще нормально, але вже на межі")
else:
    print("⚠️ Продовжуйте експерименти з різними значеннями")

print("\n" + "="*60 + "\n")

# Графік порівняння
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

categories = ['Train Accuracy', 'Test Accuracy']
values = [train_acc_hw * 100, test_acc_hw * 100]
colors = ['#3498db', '#e74c3c']

bars = ax.bar(categories, values, color=colors, alpha=0.7, width=0.5)

# Додаємо значення на стовпчики
for bar, value in zip(bars, values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 1,
            f'{value:.1f}%',
            ha='center', va='bottom', fontsize=14, fontweight='bold')

ax.set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
ax.set_title(f'Результати для max_depth = {max_depth_to_test}',
             fontsize=14, fontweight='bold', pad=15)
ax.set_ylim(0, 105)
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Додаємо анотацію залежно від результату
if max_depth_to_test is not None and max_depth_to_test <= 2:
    ax.text(0.5, 50, f'❌ UNDERFITTING\nОбидва показники низькі',
            ha='center', fontsize=11, color='blue', fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
elif max_depth_to_test is not None and max_depth_to_test >= 12:
    ax.text(0.5, 50, f'❌ OVERFITTING\nВелика різниця: {diff_hw*100:.1f}%',
            ha='center', fontsize=11, color='red', fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
elif max_depth_to_test is not None and 4 <= max_depth_to_test <= 8:
    ax.text(0.5, 50, f'✅ GOOD FIT\nРізниця всього {diff_hw*100:.1f}%',
            ha='center', fontsize=11, color='green', fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

plt.tight_layout()
plt.show()

print("\n🎯 ПОДАЛЬШІ КРОКИ:")
print("   1. Змініть max_depth_to_test на інше значення")
print("   2. Запустіть цю комірку ще раз")
print("   3. Порівняйте результати та знайдіть найкращий варіант!\n")
print("\n" + "="*80)

print("✅ ЯК ТІЛЬКИ ВІДНАЙДЕНЕ ОПТИМАЛЬНЕ ЗНАЧЕННЯ, ЗАВДАННЯ ВВАЖАЄТЬСЯ ЗАВЕРШЕНИМ!")
print("="*80)
print("""
📚 ЩО МИ ВИВЧИЛИ:

1. OVERFITTING (Перенавчання):
   • Модель "запам'ятовує" тренувальні дані
   • Висока точність на train, низька на test
   • Причини: мало даних або занадто складна модель

2. UNDERFITTING (Недонавчання):
   • Модель занадто проста
   • Низька точність на train і на test
   • Причини: погані ознаки або занадто проста модель

3. GOOD FIT (Оптимальна модель):
   • Модель добре узагальнює
   • Висока точність на train і на test
   • Мала різниця між train та test accuracy

🎯 Виконайте домашнє завдання щоб закріпити знання!
   Експериментуйте з параметрами та знайдіть найкращу модель!
""")